In [4]:
!python yolov9/detect.py \
--img 640 --conf 0.1 --device 0 \
--weights yolov9/runs/weights/best.pt \
--source test.jpg

/home/zeus/miniconda3/envs/cloudspace/bin/python: can't open file '/teamspace/studios/this_studio/yolov9/yolov9/detect.py': [Errno 2] No such file or directory


In [31]:
from ultralytics import YOLO

In [32]:
model = YOLO("yolov9c.pt")

100%|██████████| 49.4M/49.4M [00:00<00:00, 215MB/s]


In [35]:
from models.common import DetectMultiBackend
device = torch.device('cpu')
dnn = False
ROOT = '/teamspace/studios/this_studio/yolov9'
data = '/teamspace/studios/this_studio/yolov9data/coco.yaml'
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=True)

Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs


In [36]:
def predict(chosen_model, img, classes=[], conf=0.5):
    if classes:
        results = chosen_model.predict(img, classes=classes, conf=conf)
    else:
        results = chosen_model.predict(img, conf=conf)

    return results

def predict_and_detect(chosen_model, img, classes=[], conf=0.5, rectangle_thickness=2, text_thickness=1):
    results = predict(chosen_model, img, classes, conf=conf)
    for result in results:
        for box in result.boxes:
            cv2.rectangle(img, (int(box.xyxy[0][0]), int(box.xyxy[0][1])),
                          (int(box.xyxy[0][2]), int(box.xyxy[0][3])), (255, 0, 0), rectangle_thickness)
            cv2.putText(img, f"{result.names[int(box.cls[0])]}",
                        (int(box.xyxy[0][0]), int(box.xyxy[0][1]) - 10),
                        cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), text_thickness)
    return img, results

In [38]:
import cv2
image = cv2.imread("/teamspace/studios/this_studio/test.jpg")
result_img, _ = predict_and_detect(model, image, classes=[], conf=0.5)

AttributeError: 'DetectMultiBackend' object has no attribute 'predict'

In [5]:
print(os.getcwd())

/teamspace/studios/this_studio/yolov9


In [6]:
os.chdir('/teamspace/studios/this_studio')

In [7]:
print(os.getcwd())

/teamspace/studios/this_studio


In [13]:
!python /teamspace/studios/this_studio/yolov9/detect.py \
--img 640 --conf 0.1 --device 0 \
--weights /teamspace/studios/this_studio/yolov9/weights/best.pt \
--source "/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations/samples/india_psidguaj (guava)2020.10.30.15.45.36_25.237648598849773_79.32546760886908_53c529cf-f035-41e8-a90f-c3b159afef4c_img_20201030_153234_1625392956074806556.jpg"


detect: weights=['/teamspace/studios/this_studio/yolov9/weights/best.pt'], source=/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations/samples/india_psidguaj (guava)2020.10.30.15.45.36_25.237648598849773_79.32546760886908_53c529cf-f035-41e8-a90f-c3b159afef4c_img_20201030_153234_1625392956074806556.jpg, data=yolov9/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov9/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs
image 1/1 /teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations/samples/india_psidguaj (guava)2020.10.30.15.45.36_25.237648598849773_79.32546760886908_53c529cf-f035-41e

In [29]:
import argparse
import os
import platform
import sys
from pathlib import Path

import torch

def setup_yolo_env(file_path):
    FILE = Path(file_path).resolve()
    ROOT = FILE.parents[0]  # YOLO root directory
    if str(ROOT) not in sys.path:
        sys.path.append(str(ROOT))  # add ROOT to PATH
    ROOT = Path(os.path.relpath(ROOT, Path.cwd())) 
    from models.common import DetectMultiBackend
    from utils.general import (LOGGER, check_file, check_img_size, check_requirements, cv2,
                               increment_path, non_max_suppression, strip_optimizer, xyxy2xywh)
    from utils.torch_utils import select_device, smart_inference_mode

# Define the detect_yolo function
def detect_yolo(file_path, weights, source, img=640, device='cpu', conf_thres=0.25, iou_thres=0.45, max_det=1000):
    FILE = Path(file_path).resolve()
    ROOT = FILE.parents[0]  # YOLO root directory
    if str(ROOT) not in sys.path:
        sys.path.append(str(ROOT))  # add ROOT to PATH
    ROOT = Path(os.path.relpath(ROOT, Path.cwd())) 
    from models.common import DetectMultiBackend
    from utils.general import (LOGGER, check_file, check_img_size, check_requirements, cv2,
                               increment_path, non_max_suppression, strip_optimizer, xyxy2xywh)
    from utils.torch_utils import select_device, smart_inference_mode


    @smart_inference_mode()
    def run_detection():
        source_str = str(source)
        save_img = False  # save inference images
        nosave = True  # do not save images/videos
        is_file = Path(source_str).suffix[1:] in ('jpg', 'jpeg', 'png', 'bmp', 'tif', 'tiff')  # image formats
        is_url = source_str.lower().startswith(('http://', 'https://'))
        webcam = source_str.isnumeric() or source_str.endswith('.txt') or (is_url and not is_file)
        screenshot = source_str.lower().startswith('screen')
        if is_url and is_file:
            source_str = check_file(source_str)  # download

        # Load model
        #device = select_device(device)
        #weights = check_file(weights)  # download if not found locally
        device = torch.device('cpu')
        model = DetectMultiBackend(weights, device=device)
        stride, names, pt = model.stride, model.names, model.pt
        imgsz = check_img_size(img, s=stride)  # check image size

        # Dataloader
        dataset = [source]
        #dataset = LoadImages(source_str, img_size=imgsz, stride=stride, auto=pt)

        # Run inference
        model.warmup(imgsz=(1, 3, 640, 640))  # warmup
        #seen, windows, dt = 0, [], (Profile(), Profile(), Profile())
        results = []
        im = cv2.imread(source)
        im = torch.from_numpy(im).to(model.device)
        im = im.half() if model.fp16 else im.float()
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None] 
        pred = model(im)
        pred = pred[0][1] if isinstance(pred[0], list) else pred[0]
        pred = non_max_suppression(pred, conf_thres, iou_thres, max_det=max_det)
        pred = non_max_suppression(pred, conf_thres, iou_thres, max_det=max_det)
        for i, det in enumerate(pred):  # per image
            seen += 1
            p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)
            p = Path(p)  # to Path
            save_path = None  # don't save
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()
        
                # Collect results
                for *xyxy, conf, cls in reversed(det):
                    result = {'bbox': xyxy, 'confidence': conf, 'class': cls, 'label': names[int(cls)]}
                    results.append(result)
        LOGGER.info(f"{s}{'' if len(det) else '(no detections), '}{dt[1].dt * 1E3:.1f}ms")
        return results
    '''
        for path, im, im0s, vid_cap, s in dataset:
            with dt[0]:
                im = torch.from_numpy(im).to(model.device)
                im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
                im /= 255  # 0 - 255 to 0.0 - 1.0
                if len(im.shape) == 3:
                    im = im[None]  # expand for batch dim

            # Inference
            with dt[1]:
                pred = model(im)

            # NMS
            with dt[2]:
                pred = pred[0][1] if isinstance(pred[0], list) else pred[0]
                pred = non_max_suppression(pred, conf_thres, iou_thres, max_det=max_det)

            # Process predictions
            for i, det in enumerate(pred):  # per image
                seen += 1
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)
                p = Path(p)  # to Path
                save_path = None  # don't save
                s += '%gx%g ' % im.shape[2:]  # print string
                gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
                if len(det):
                    # Rescale boxes from img_size to im0 size
                    det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

                    # Collect results
                    for *xyxy, conf, cls in reversed(det):
                        result = {'bbox': xyxy, 'confidence': conf, 'class': cls, 'label': names[int(cls)]}
                        results.append(result)

            # Print time (inference-only)
            LOGGER.info(f"{s}{'' if len(det) else '(no detections), '}{dt[1].dt * 1E3:.1f}ms")

        return results
        
    '''
    results = run_detection()
    return results
    
# Example usage in a notebook
# Specify your parameters
weights = '/teamspace/studios/this_studio/yolov9/weights/best.pt'
source = '/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations/samples/india_psidguaj (guava)2020.10.30.15.45.36_25.237648598849773_79.32546760886908_53c529cf-f035-41e8-a90f-c3b159afef4c_img_20201030_153234_1625392956074806556.jpg'
img = 640
device = 'cpu'  # or 'cpu'
conf_thres = 0.25
iou_thres = 0.45
max_det = 1000

# Call the function
results = detect_yolo('yolov9/Untitled.ipynb',weights, source, img, device, conf_thres, iou_thres, max_det)

# Display results
print(results)


Fusing layers... 
gelan-c summary: 467 layers, 25411731 parameters, 0 gradients, 102.5 GFLOPs


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[1, 512, 512, 3] to have 3 channels, but got 512 channels instead

In [16]:
    FILE = Path('yolov9/Untitled.ipynb').resolve()
    ROOT = FILE.parents[0]  

In [17]:
print(ROOT)

/teamspace/studios/this_studio/yolov9
